# Pandas tips

In [1]:
# Core libraries
import numpy as np
import pandas as pd

# Machine Learning
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

## Functions

In [2]:
def report_datashape(X_train, X_test, y_train, y_test):
    print(f"X train: {X_train.shape}")
    print(f"X test: {X_test.shape}")
    print(f"y train: {y_train.shape}")
    print(f"y test: {y_test.shape}")
    return

## Load data

In [3]:
X, y = load_iris(return_X_y=True, as_frame=True)

## Split data into train and test

In [4]:
# Standard method of splitting data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [5]:
print("Iris dataset split using scikit learn")
report_datashape(X_train, X_test, y_train, y_test)

Iris dataset split using scikit learn
X train: (112, 4)
X test: (38, 4)
y train: (112,)
y test: (38,)


In [6]:
# Pandas method of splitting data into train and test
X_train = X.sample(frac=0.75, random_state=42)
X_test = X.sample(frac=0.25, random_state=42)
y_train = y.sample(frac=0.75, random_state=42)
y_test = y.sample(frac=0.25, random_state=42)

In [7]:
print("Iris dataset split using Pandas")
report_datashape(X_train, X_test, y_train, y_test)

Iris dataset split using Pandas
X train: (112, 4)
X test: (38, 4)
y train: (112,)
y test: (38,)


## Binning data

* Group (bin) data into multiple buckets
* pandas.qcut()
  * Divides the underlying data into equal sized bins
  * Bins are defined using percentiles based on the distribution of data
  * Bins are not defined on the actual numeric edges of the bins

In [8]:
X_train.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
73,6.1,2.8,4.7,1.2
18,5.7,3.8,1.7,0.3
118,7.7,2.6,6.9,2.3
78,6.0,2.9,4.5,1.5
76,6.8,2.8,4.8,1.4


In [9]:
pd.qcut(X_train['sepal width (cm)'], q=5).value_counts()

(2.7, 3.0]      33
(1.999, 2.7]    25
(3.4, 4.4]      20
(3.0, 3.2]      19
(3.2, 3.4]      15
Name: sepal width (cm), dtype: int64

## Slicing data

In [10]:
X.loc[100:105, 'petal length (cm)':'petal width (cm)']

,petal length (cm),petal width (cm)
100,6.0,2.5
101,5.1,1.9
102,5.9,2.1
103,5.6,1.8
104,5.8,2.2
105,6.6,2.1


In [11]:
X.iloc[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2


## Mean imputation and interpolate method

In [12]:
# Check data for missing values - none missing
X.isnull().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
dtype: int64

In [13]:
# Artifically create a missing value
X.loc[2, 'sepal width (cm)'] = np.nan

In [14]:
X.iloc[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,NaN,1.3,0.2
3,4.6,3.1,1.5,0.2


In [15]:
# Check data for missing values - one value missing
X.isnull().sum()

sepal length (cm)    0
sepal width (cm)     1
petal length (cm)    0
petal width (cm)     0
dtype: int64

In [16]:
# Replace missing values with the mean of the feature column
# Note that .interpolate() does not handle the case where the first row has a NaN as it only fills forward
X['sepal width (cm)'].fillna(X['sepal width (cm)'].mean(), inplace=True)

In [17]:
X.iloc[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.500000,1.4,0.2
1,4.9,3.000000,1.4,0.2
2,4.7,3.056376,1.3,0.2
3,4.6,3.100000,1.5,0.2


In [18]:
# Artifically create a missing value
X.loc[2, 'sepal width (cm)'] = np.nan

In [19]:
X.interpolate()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.50,1.4,0.2
1,4.9,3.00,1.4,0.2
2,4.7,3.05,1.3,0.2
3,4.6,3.10,1.5,0.2
4,5.0,3.60,1.4,0.2
...,...,...,...,...
145,6.7,3.00,5.2,2.3
146,6.3,2.50,5.0,1.9
147,6.5,3.00,5.2,2.0
148,6.2,3.40,5.4,2.3


In [20]:
# Use interpolation to impute missing values
X['sepal width (cm)'].fillna(X['sepal width (cm)'].interpolate(), inplace=True)

In [21]:
# Check data for missing values - one value missing
X.isnull().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
dtype: int64

In [22]:
X.iloc[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.50,1.4,0.2
1,4.9,3.00,1.4,0.2
2,4.7,3.05,1.3,0.2
3,4.6,3.10,1.5,0.2


## Combining data

### Concatenation

In [27]:
# Concatenate X and y DataFrames
iris_df = pd.concat([X, y], axis=1)
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.50,1.4,0.2,0
1,4.9,3.00,1.4,0.2,0
2,4.7,3.05,1.3,0.2,0
3,4.6,3.10,1.5,0.2,0
4,5.0,3.60,1.4,0.2,0


### Joins

In [28]:
# Inner join
iris_inner_df = pd.merge(X_train, X_test, on='sepal length (cm)')
iris_inner_df.head()

,sepal length (cm),sepal width (cm)_x,petal length (cm)_x,petal width (cm)_x,sepal width (cm)_y,petal length (cm)_y,petal width (cm)_y
0,6.1,2.8,4.7,1.2,2.8,4.7,1.2
1,6.1,2.8,4.7,1.2,3.0,4.9,1.8
2,6.1,3.0,4.9,1.8,2.8,4.7,1.2
3,6.1,3.0,4.9,1.8,3.0,4.9,1.8
4,6.1,2.6,5.6,1.4,2.8,4.7,1.2


In [29]:
# Full outer join
iris_outer_df = pd.merge(X_train, X_test, how='outer')
iris_outer_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,6.1,2.8,4.7,1.2
1,5.7,3.8,1.7,0.3
2,7.7,2.6,6.9,2.3
3,6.0,2.9,4.5,1.5
4,6.8,2.8,4.8,1.4


In [30]:
# Left join
iris_left_df = pd.merge(X_train, X_test, how='left')
iris_left_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,6.1,2.8,4.7,1.2
1,5.7,3.8,1.7,0.3
2,7.7,2.6,6.9,2.3
3,6.0,2.9,4.5,1.5
4,6.8,2.8,4.8,1.4


In [31]:
# Right join
iris_right_df = pd.merge(X_train, X_test, how='right')
iris_right_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,6.1,2.8,4.7,1.2
1,5.7,3.8,1.7,0.3
2,7.7,2.6,6.9,2.3
3,6.0,2.9,4.5,1.5
4,6.8,2.8,4.8,1.4
